In this notebook we will be using SALES dataset stored in a mysql database and we will use SUpervised classification algorithms for learning how profit is generated and we print the accuracy, accuracy matrix for each of these..

In [2]:
import pandas as pd
import numpy as np
import sqlalchemy
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
# Import train_test_split function
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

In [3]:
database_username = 'root'#enter db admin username
database_password = 'root' #enter db password
database_ip       = '127.0.0.1:3306' #enter host connection
database_name     = 'storedata' #db name
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))
#to store the table into MySQL database...it is has been commented because the table is already stored in database
#store_df.to_sql(con=database_connection, name='storedata', if_exists='replace')

In [4]:
#we create a dataframe from the table ' storedate' in the database....
store_df = pd.read_sql("select * from storedata", database_connection);

In [5]:
store_df.describe()

,index,Sales,Quantity,Discount,Profit,Month
count,4210.000000,4210.000000,4210.000000,4210.000000,4210.000000,4210.000000
mean,2104.500000,545.653411,8.995962,0.370805,83.800475,7.125416
std,1215.466646,1063.543578,7.597797,0.536742,184.528025,3.430877
min,0.000000,0.836000,1.000000,0.000000,0.000000,1.000000
25%,1052.250000,48.817500,4.000000,0.000000,0.000000,4.000000
50%,2104.500000,198.094000,7.000000,0.200000,50.000000,7.000000
75%,3156.750000,632.679750,12.000000,0.437500,75.000000,10.000000
max,4209.000000,23661.228000,70.000000,4.000000,2500.000000,12.000000


In [6]:
#we reset the index...
store_df.set_index('index', inplace=True)

In [7]:
#change the data object to datetimetype
store_df['Order Date'] = pd.to_datetime(store_df['Order Date'])
#we convert profit to integer to make data non-continous...
store_df['Profit'] = store_df['Profit'].astype('int')

In [8]:
store_df.dtypes

State                 object
Order Date    datetime64[ns]
Sales                float64
Quantity               int64
Discount             float64
Profit                 int32
Month                  int64
dtype: object

In [9]:
#extract month from datetime so we can later use it to predict monthly profits
store_df['Month'] = pd.DatetimeIndex(store_df['Order Date']).month

In [10]:
#since ML algorithms dont work with strings we encode the 'State' column into a onehotencoded code....
x = store_df[['State']]
encoded_states = OneHotEncoder().fit_transform(x).toarray()
print(encoded_states.shape)


(4210, 49)


In [11]:
#setting features...
store_data = store_df[['Month','Sales','Quantity','Discount']].to_numpy()

In [12]:
#merging encoded category with the features...
store_data = np.concatenate((encoded_states,store_data),axis=1)

In [124]:
#since these are already in the database as integers classes...
#we make classes for level of profits....

#the store has a fixed profit for sales made within these ranges for e.g any sales between 100 to 500 
#have a fixed prfit of 50 and so other rates 500 to 1000 are 75 etc...

#store_df['Profit'].loc[store_df['Sales'].between(0, 100)] = 0


#store_df['Profit'].loc[store_df['Sales'].between(100, 500)] = 50
    

#store_df['Profit'].loc[store_df['Sales'].between(500, 1000)] = 75
    

#store_df['Profit'].loc[store_df['Sales'].between(1000, 2000)] = 200
    

#store_df['Profit'].loc[store_df['Sales'].between(2000, 5000)] = 500
        
#store_df['Profit'].loc[store_df['Sales'].between(5000, 10000)] = 1500
#store_df['Profit'].loc[store_df['Sales'].between(10000, 50000)] = 2500


In [62]:
#storing the dataframe as csv for later use....
store_df.to_csv('C:/Users/Hamza-Acer/Documents/storedata_processed.csv', index = False)

In [13]:
#setting target label....
store_target = store_df[['Profit']].to_numpy()

In [14]:
#We store the Profit column into a numpy array which is our target column...
#target column is the label we want to predict['Profit']...
store_target

array([[  0],
       [  0],
       [  0],
       ...,
       [200],
       [ 50],
       [200]])

In [15]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(store_data, store_target, test_size=0.3,random_state=109) # 70% training and 30% test

In [16]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf_SVM = svm.SVC(kernel='rbf') # rbf Kernel

#Train the model using the training sets
clf_SVM.fit(X_train, y_train)

#Predict the response for test dataset
y_pred_SVM = clf_SVM.predict(X_test)

c:\users\hamza-acer\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [17]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_SVM))

Accuracy: 0.9746634996041171


In [18]:
#predict the profit for all data...
y_pred_SVM = clf_SVM.predict(store_data)

In [118]:
#adding the predicted column to the store_df table for analysis.....
store_df['Predicted Profit'] = pd.Series(y_pred_SVM)
#save it as a CSV for tableau data analysis
store_df.to_csv('C:/Users/Hamza-Acer/Documents/storedata_SVM.csv', index = False)

In [19]:
#Print accuracy matrix for SVM kernel'rbf' algorithm for complete data...
print(confusion_matrix(store_target, y_pred_SVM))
print(classification_report(store_target, y_pred_SVM))

[[1560    0    0    0    0    0    0]
 [  94 1277    0    0    0    0    0]
 [   0   13  621    0    0    0    0]
 [   0    0    6  409    1    0    0]
 [   0    0    0    0  197    0    0]
 [   0    0    0    0    3   22    0]
 [   0    0    0    0    0    1    6]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1560
          50       0.99      0.93      0.96      1371
          75       0.99      0.98      0.98       634
         200       1.00      0.98      0.99       416
         500       0.98      1.00      0.99       197
        1500       0.96      0.88      0.92        25
        2500       1.00      0.86      0.92         7

    accuracy                           0.97      4210
   macro avg       0.98      0.95      0.96      4210
weighted avg       0.97      0.97      0.97      4210



In [20]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier(n_neighbors=7)# we have 7 categories of profits
clf_knn.fit(X_train, y_train)

y_pred_knn = clf_knn.predict(X_test)

<ipython-input-20-6a0797e61d43>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  clf_knn.fit(X_train, y_train)


In [21]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_knn))

Accuracy: 0.9936658749010293


In [22]:
#predict the profit for all data...
y_pred_knn = clf_knn.predict(store_data)

In [113]:
#adding the predicted column to the store_df table for analysis.....
store_df['Predicted Profit'] = pd.Series(y_pred_knn)
#save it as a CSV for tableau data analysis
store_df.to_csv('C:/Users/Hamza-Acer/Documents/storedata_knn.csv', index = False)

In [23]:
#Print accuracy matrix for KNN algorithm for complete data...
print(confusion_matrix(store_target, y_pred_knn))
print(classification_report(store_target, y_pred_knn))

[[1557    3    0    0    0    0    0]
 [   2 1367    2    0    0    0    0]
 [   0    0  632    2    0    0    0]
 [   0    0    1  414    1    0    0]
 [   0    0    0    0  197    0    0]
 [   0    0    0    0    4   21    0]
 [   0    0    0    0    0    2    5]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1560
          50       1.00      1.00      1.00      1371
          75       1.00      1.00      1.00       634
         200       1.00      1.00      1.00       416
         500       0.98      1.00      0.99       197
        1500       0.91      0.84      0.87        25
        2500       1.00      0.71      0.83         7

    accuracy                           1.00      4210
   macro avg       0.98      0.93      0.95      4210
weighted avg       1.00      1.00      1.00      4210



In [24]:
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
clf_logreg = LogisticRegression()

# fit the model with data
clf_logreg.fit(X_train,y_train)

#
y_pred_logreg=clf_logreg.predict(X_test)

c:\users\hamza-acer\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\users\hamza-acer\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
# Model Accuracy: for the test data
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_logreg))

Accuracy: 0.7410926365795725


In [26]:
#predict the profit for all data...
y_pred_logreg = clf_logreg.predict(store_data)

In [106]:
#adding the predicted column to the store_df table for analysis.....
store_df['Predicted Profit'] = pd.Series(y_pred_logreg)
#save it as a CSV for tableau data analysis
store_df.to_csv('C:/Users/Hamza-Acer/Documents/storedata_logreg.csv', index = False)

In [27]:
#accuracy of using logarithmic regression algorithm for the complete dataset
print(confusion_matrix(store_target, y_pred_logreg))
print(classification_report(store_target, y_pred_logreg))

[[1356  203    1    0    0    0    0]
 [  85 1069  174   41    2    0    0]
 [   0  136  377  108   13    0    0]
 [   0    3  144  211   55    3    0]
 [   0    0    0   86  103    8    0]
 [   0    0    0    0   18    7    0]
 [   0    0    0    0    1    6    0]]
              precision    recall  f1-score   support

           0       0.94      0.87      0.90      1560
          50       0.76      0.78      0.77      1371
          75       0.54      0.59      0.57       634
         200       0.47      0.51      0.49       416
         500       0.54      0.52      0.53       197
        1500       0.29      0.28      0.29        25
        2500       0.00      0.00      0.00         7

    accuracy                           0.74      4210
   macro avg       0.51      0.51      0.51      4210
weighted avg       0.75      0.74      0.75      4210



c:\users\hamza-acer\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
from sklearn.tree import DecisionTreeClassifier
# Create Decision Tree classifer object
clf_tree = DecisionTreeClassifier(criterion="entropy", max_depth=4)

# Train Decision Tree Classifer
clf_tree = clf_tree.fit(X_train,y_train)

#Predict the response for test dataset
y_pred_tree = clf_tree.predict(X_test)

In [29]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_tree))

Accuracy: 0.9920823436262867


In [30]:
y_pred_tree = clf_tree.predict(store_data)

In [100]:
#adding the predicted column to the store_df table for analysis.....
store_df['Predicted Profit'] = pd.Series(y_pred_tree)
#save it as a CSV for tableau data analysis
store_df.to_csv('C:/Users/Hamza-Acer/Documents/storedata_tree.csv', index = False)

In [31]:
#accuracy of using tree algorithm for the complete dataset
print(confusion_matrix(store_target, y_pred_tree))
print(classification_report(store_target, y_pred_tree))

[[1560    0    0    0    0    0    0]
 [   0 1370    1    0    0    0    0]
 [   0    0  634    0    0    0    0]
 [   0    0    1  415    0    0    0]
 [   0    0    0    0  197    0    0]
 [   0    0    0    0   25    0    0]
 [   0    0    0    0    7    0    0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1560
          50       1.00      1.00      1.00      1371
          75       1.00      1.00      1.00       634
         200       1.00      1.00      1.00       416
         500       0.86      1.00      0.92       197
        1500       0.00      0.00      0.00        25
        2500       0.00      0.00      0.00         7

    accuracy                           0.99      4210
   macro avg       0.69      0.71      0.70      4210
weighted avg       0.99      0.99      0.99      4210



c:\users\hamza-acer\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
print("Accuracy of SVM",metrics.accuracy_score(store_target, y_pred_SVM))
print("Accuracy of KNN",metrics.accuracy_score(store_target, y_pred_knn))
print("Accuracy of Logarithmic Regression",metrics.accuracy_score(store_target, y_pred_logreg))
print("Accuracy of Decision Tree Classifier",metrics.accuracy_score(store_target, y_pred_tree))

Accuracy of SVM 0.9719714964370546
Accuracy of KNN 0.9959619952494062
Accuracy of Logarithmic Regression 0.7418052256532066
Accuracy of Decision Tree Classifier 0.9919239904988123
